In [1]:
import re
import pandas
import pymorphy2
from functools import reduce
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np

# Очистка данных

В качестве данных использованы вопросы по Computer Science

In [2]:
questions = pandas.read_excel('questions.xlsx')
questions.head()

,question,answer
0,Когда были созданы первые ЭВМ?,В 40-е годы XX века.
1,Кто является первым программистом?,Ада Лавлейс
2,Что такое компилятор?,"Программа, перобразующая программы, написанные..."
3,Что такое интерпретатор?,"Программа, выполняющая построчный анализ, обра..."
4,Какие существуют этапы компиляции?,"Лекчический анализ, синтаксический анализ, сем..."


Для токинизации предложения необходимо:
- убрать знаки препинания
- убрать лишние пробельные символы
- привести к одному регистру
- разделить

In [124]:
punctuation_pattern = re.compile('[?]')

def tokenize_sentence(sentence):
    return punctuation_pattern.sub('', sentence).strip().lower().split()

def tokenize_sentences(sentences):
    tokens = reduce(lambda a, x: a + x, map(lambda x: tokenize_sentence(x), sentences))
    return tokens

def tokenize_sentences_with_count(tokensLocal):
    uniqueTokens = set(tokensLocal)
    uniqueTokensWithCount = list(map(lambda x: (x, tokensLocal.count(x)), uniqueTokens))
    sortedUniqueTokensWithCount = sorted(uniqueTokensWithCount, key=lambda x: x[1], reverse=True)
    return sortedUniqueTokensWithCount
    

In [125]:
tokens = tokenize_sentences(questions["question"])
tokensWithCount = tokenize_sentences_with_count(tokens)
tokensWithCount

[('что', 58),
 ('такое', 57),
 ('какие', 16),
 ('бывают', 14),
 ('уровня', 9),
 ('протоколы', 7),
 ('уровень', 5),
 ('системы', 3),
 ('существуют', 2),
 ('как', 2),
 ('процесс', 2),
 ('система', 2),
 ('модели', 2),
 ('представления', 2),
 ('память', 2),
 ('сеть', 2),
 ('протокол', 2),
 ('сетевые', 2),
 ('сетевой', 2),
 ('прикладного', 2),
 ('псевдокод', 1),
 ('взаимодействие', 1),
 ('первые', 1),
 ('сокет', 1),
 ('файл', 1),
 ('компиляции', 1),
 ('отладчик', 1),
 ('физического', 1),
 ('этапы', 1),
 ('мультиплдексор', 1),
 ('компилятор', 1),
 ('дерево', 1),
 ('транспортный', 1),
 ('отладчики', 1),
 ('системные', 1),
 ('файловая', 1),
 ('эвм', 1),
 ('состоит', 1),
 ('сеансового', 1),
 ('модель', 1),
 ('компьютерная', 1),
 ('является', 1),
 ('типы', 1),
 ('операционной', 1),
 ('наследование', 1),
 ('кольцо', 1),
 ('регистр', 1),
 ('полиморфизм', 1),
 ('компоновщик', 1),
 ('канальный', 1),
 ('вызовы', 1),
 ('мьютекс', 1),
 ('интернет', 1),
 ('многозадачность', 1),
 ('страниц', 1),
 ('гонки

Для лемматизации понадобится преобразовать каждое слово в словарную форму и заново рассчитать количества вхождений.

В результате мы увидим, что некоторые слова вроде "уровня" и "уровень" объединились, а их количества суммировались

In [130]:
def lemmatize_tokents_with_count(tokensWithCountLocal):
    morph = pymorphy2.MorphAnalyzer()

    repeatingLemmasWithCount = list(map(lambda x: (morph.parse(x[0])[0].normal_form, x[1]), tokensWithCountLocal))
    uniqueLemmas = set(map(lambda x: x[0], repeatingLemmasWithCount))
    lemmasWithCount = list(map(lambda x: (x, reduce(lambda a, x: a + x, map(lambda z: z[1], filter(lambda y: y[0]==x, repeatingLemmasWithCount)))), uniqueLemmas))
    sortedLemmasWithCount = sorted(lemmasWithCount, key=lambda x: x[1], reverse=True)
    return sortedLemmasWithCount

lemmasWithCount = lemmatize_tokents_with_count(tokensWithCount)
lemmasWithCountDict = dict(lemmasWithCount)
lemmasWithCount

[('что', 59),
 ('такой', 57),
 ('какой', 16),
 ('уровень', 15),
 ('бывать', 14),
 ('протокол', 9),
 ('сетевой', 6),
 ('система', 5),
 ('модель', 3),
 ('операционный', 3),
 ('один', 3),
 ('отладчик', 2),
 ('как', 2),
 ('транспортный', 2),
 ('процесс', 2),
 ('канальный', 2),
 ('прикладной', 2),
 ('файловый', 2),
 ('сеансовый', 2),
 ('память', 2),
 ('представление', 2),
 ('дизассемблер', 2),
 ('существовать', 2),
 ('сеть', 2),
 ('физический', 2),
 ('псевдокод', 1),
 ('взаимодействие', 1),
 ('сокет', 1),
 ('файл', 1),
 ('программирование', 1),
 ('программист', 1),
 ('межпроцессный', 1),
 ('мультиплдексор', 1),
 ('компилятор', 1),
 ('дерево', 1),
 ('ала', 1),
 ('эвм', 1),
 ('наследование', 1),
 ('кольцо', 1),
 ('называться', 1),
 ('регистр', 1),
 ('вызов', 1),
 ('полиморфизм', 1),
 ('процессор', 1),
 ('компоновщик', 1),
 ('разделять', 1),
 ('мьютекс', 1),
 ('интернет', 1),
 ('многозадачность', 1),
 ('шина', 1),
 ('когда', 1),
 ('html', 1),
 ('из', 1),
 ('состояние', 1),
 ('сумматор', 1),
 (

Для анализа выбросов можно отобразить полученные данные на графиках

In [119]:
init_notebook_mode(connected=True)

Гистограмма

In [128]:
iplot([go.Bar(x=list(lemmasWithCountDict.keys()), y=list(lemmasWithCountDict.values()))])

In [129]:
iplot([go.Histogram(x=tokens, xbins=dict(size=10))])

Ящик с усами чётко показывает выбросы

In [141]:
data_box = [go.Box(y=list(lemmasWithCountDict.values()),
                   boxpoints='all',
                   pointpos=-1.5,
                   name='Count')]
layout_box = go.Layout(boxgap=0.8)

iplot(go.Figure(data=data_box, layout=layout_box))

Межквартальный размах убрал нужное и оставил лишнее

In [153]:
Q1, Q2, Q3 = np.percentile(list(map(float, lemmasWithCountDict.values())), [25, 50, 75])
IQR = Q3 - Q1
lower_inner_fence = Q1 - (1.5 * IQR)
lower_outer_fence = Q1 - (3 * IQR)
upper_inner_fence = Q3 + (1.5 * IQR)
upper_outer_fence = Q3 + (3 * IQR)
print(f'''upper_outer_fence:  {upper_outer_fence}, 
upper_inner_fence:  {upper_inner_fence},
lower_inner_fence:  {lower_inner_fence},
lower_outer_fence"  {lower_outer_fence}''')
print('counts with outlier   ', list(lemmasWithCountDict.values()))
print('counts without outlier', list(filter(lambda x: lower_inner_fence < x < upper_inner_fence, lemmasWithCountDict.values())))

upper_outer_fence:  5.0, 
upper_inner_fence:  3.5,
lower_inner_fence:  -0.5,
lower_outer_fence"  -2.0
counts with outlier    [59, 57, 16, 15, 14, 9, 6, 5, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
counts without outlier [3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Таким образом, стоп-слова логично отобрать исходя из результатов графического анализа

In [157]:
stopWords = list(map(lambda x: x[0],filter(lambda x: x[1] > 10, lemmasWithCount)))
stopWords

['что', 'такой', 'какой', 'уровень', 'бывать']

Итоговый список подходящих лемм

In [161]:
lemmasWithoutStopWords = list(map(lambda x: x[0], filter(lambda x: x[0] not in stopWords, lemmasWithCount)))
lemmasWithoutStopWords

['протокол',
 'сетевой',
 'система',
 'модель',
 'операционный',
 'один',
 'отладчик',
 'как',
 'транспортный',
 'процесс',
 'канальный',
 'прикладной',
 'файловый',
 'сеансовый',
 'память',
 'представление',
 'дизассемблер',
 'существовать',
 'сеть',
 'физический',
 'псевдокод',
 'взаимодействие',
 'сокет',
 'файл',
 'программирование',
 'программист',
 'межпроцессный',
 'мультиплдексор',
 'компилятор',
 'дерево',
 'ала',
 'эвм',
 'наследование',
 'кольцо',
 'называться',
 'регистр',
 'вызов',
 'полиморфизм',
 'процессор',
 'компоновщик',
 'разделять',
 'мьютекс',
 'интернет',
 'многозадачность',
 'шина',
 'когда',
 'html',
 'из',
 'состояние',
 'сумматор',
 'хеширование',
 'являться',
 'поток',
 'ассемблер',
 'подкачка',
 'бибилиотека',
 'создать',
 'api',
 'тип',
 'обранизоваться',
 'машина',
 'состоять',
 'системный',
 'высокоуровневый',
 'страница',
 'алгоритм',
 'этап',
 'сообщение',
 'компиляция',
 'компьютерный',
 'гонка',
 'язык',
 'семафор',
 'дескриптор',
 'звезда',
 'микроп